# Imports

In [1]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers requests beautifulsoup4 pandas numpy
!pip install text2emotion

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.7 MB/s eta 0:11:06tcmalloc: large alloc 1147494400 bytes == 0x556b4c306000 @  0x7fdf4055c615 0x556b1291d4cc 0x556b129fd47a 0x556b129202ed 0x556b12a11e1d 0x556b12993e99 0x556b1298e9ee 0x556b12921bda 0x556b12993d00 0x556b1298e9ee 0x556b12921bda 0x556b12990737 0x556b12a12c66 0x556b1298fdaf 0x556b12a12c66 0x556b1298fdaf 0x556b12a12c66 0x556b1298fdaf 0x556b12922039 0x556b12965409 0x556b12920c52 0x556b12993c25 0x556b1298e9ee 0x556b12921bda 0x556b12990737 0x556b1298e9ee 0x556b12921bda 0x556b1298f915 0x556b12921afa 0x556b1298fc0d 0x556b1298e9ee
     |█████████████████               | 1055.7 MB 1.6 MB/s eta 0:09:52tcmalloc: large alloc 1434370048 bytes == 0x556b9095c000 @  0x7fdf4055c615 0x556b1291d4cc 0x556b129fd47a 0x556b129202ed 0x556b12a11e1d 0x556b12993e99 0x556b1298e9ee 0x556b12921bda 0x556b12993d00 0x556b1298e9ee 0x556b12921bda 0x556b12990737 0x556b12a12c66 0x556b1298fd

In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy

import text2emotion as te
from nltk.sentiment import SentimentIntensityAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import files
from datetime import datetime

# DataFrame Loading

In [ ]:
df_chevy = pd.read_json("/content/drive/MyDrive/quotes/Chevy-quotes-2015.json")
df_chevy = df_chevy.append(pd.read_json("/content/drive/MyDrive/quotes/Chevy-quotes-2016.json"))
df_chevy = df_chevy.append(pd.read_json("/content/drive/MyDrive/quotes/Chevy-quotes-2017.json"))
df_chevy = df_chevy.append(pd.read_json("/content/drive/MyDrive/quotes/Chevy-quotes-2018.json"))
df_chevy = df_chevy.append(pd.read_json("/content/drive/MyDrive/quotes/Chevy-quotes-2019.json"))
df_chevy = df_chevy.append(pd.read_json("/content/drive/MyDrive/quotes/Chevy-quotes-2020.json"))
df_chevy.reset_index(inplace=True)

df_tesla = pd.read_json("/content/drive/MyDrive/quotes/Tesla-quotes-2015.json")
df_tesla = df_tesla.append(pd.read_json("/content/drive/MyDrive/quotes/Tesla-quotes-2016.json"))
df_tesla = df_tesla.append(pd.read_json("/content/drive/MyDrive/quotes/Tesla-quotes-2017.json"))
df_tesla = df_tesla.append(pd.read_json("/content/drive/MyDrive/quotes/Tesla-quotes-2018.json"))
df_tesla = df_tesla.append(pd.read_json("/content/drive/MyDrive/quotes/Tesla-quotes-2019.json"))
df_tesla = df_tesla.append(pd.read_json("/content/drive/MyDrive/quotes/Tesla-quotes-2020.json"))
df_tesla.reset_index(inplace=True)

# NLP Pipeline

In [ ]:
def clean(text):

    text = re.sub('[^A-Za-z]+', ' ', text)
    return text

def listToString(s): 
  
    str1 = " " 
    return (str1.join(s))

def processing(reviews):

  tokenized = []
  stop_rmvd_string = []

  for sentence in reviews:
    
    tokens = [token.text for token in nlp(sentence)]
    tokenized.append(tokens)

    #pos_tag = [(token.text, token.pos_) for token in nlp(sentence)]
    #pos_tagged.append(pos_tag)

    stop_words = [token.text for token in nlp(sentence) if not token.is_stop]
    #stop_rmvd.append(stop_words)
    stop_rmvd_string.append(listToString(stop_words))

    #lemma = [token.lemma_ for token in nlp(listToString(stop_words)) if token.text != token.lemma_]
    #lemmas.append(lemma)

  return tokenized, stop_rmvd_string

nlp = spacy.load('en')

In [ ]:
df_tesla['Cleaned Reviews'] = df_tesla['quotation'].apply(clean)

tesla_tokenized, tesla_rmvd_string = processing(df_tesla["Cleaned Reviews"])
df_tesla['Tokenized'] = tesla_tokenized
df_tesla['Stop Words Removed'] = tesla_rmvd_string

#df_cleaning['POS Tagged'] = pos_tagged
#df_cleaning['Stop Words'] = stop_rmvd
#df_cleaning['Lemma'] = lemmas

df_chevy['Cleaned Reviews'] = df_chevy['quotation'].apply(clean)

chevy_tokenized, chevy_rmvd_string = processing(df_chevy["Cleaned Reviews"])
df_chevy['Tokenized'] = chevy_tokenized
df_chevy['Stop Words Removed'] = chevy_rmvd_string

In [ ]:
df_tesla

,index,date,qids,quotation,quoteID,Cleaned Reviews,Tokenized,Stop Words Removed
0,0,2015-04-17 17:55:00,Q6751189,The policy will concentrate on `A' category st...,76279,The policy will concentrate on A category stat...,"[The, policy, will, concentrate, on, A, catego...",policy concentrate category stations developed...
1,1,2015-10-29 16:31:31,Q4934,As Sergey and I wrote in the original founders...,10100,As Sergey and I wrote in the original founders...,"[As, Sergey, and, I, wrote, in, the, original,...",Sergey wrote original founders letter years ag...
2,2,2015-10-14 10:18:21,Q317521,Elon Musk: Apple hired the Tesla engineers we'...,22047,Elon Musk Apple hired the Tesla engineers we v...,"[Elon, Musk, Apple, hired, the, Tesla, enginee...",Elon Musk Apple hired Tesla engineers ve fired
3,3,2015-05-21 18:50:00,Q21456834,"If Tesla folds tomorrow, there will be a few h...",28626,If Tesla folds tomorrow there will be a few hu...,"[If, Tesla, folds, tomorrow, there, will, be, ...",Tesla folds tomorrow people Connecticut disapp...
4,4,2015-10-12 02:46:53,Q29643422,This is the best racing on the planet in my op...,90436,This is the best racing on the planet in my op...,"[This, is, the, best, racing, on, the, planet,...",best racing planet opinion Late Model Stock Ca...
...,...,...,...,...,...,...,...,...
19575,937,2020-02-19 09:18:25,Q317521,Gigafactory Berlin-Brandenburg is the next pha...,20355,Gigafactory Berlin Brandenburg is the next pha...,"[Gigafactory, Berlin, Brandenburg, is, the, ne...",Gigafactory Berlin Brandenburg phase Tesla s p...
19576,938,2020-01-15 13:46:36,Q125060,The state of California requires a lemon vehic...,84107,The state of California requires a lemon vehic...,"[The, state, of, California, requires, a, lemo...",state California requires lemon vehicle meet c...
19577,939,2020-03-19 00:00:00,Q317521,Tesla makes cars with sophisticated hvac syste...,59397,Tesla makes cars with sophisticated hvac syste...,"[Tesla, makes, cars, with, sophisticated, hvac...",Tesla makes cars sophisticated hvac systems Sp...
19578,940,2020-04-13 00:00:00,Q4964201,There's the space of the Model S on the inside...,50092,There s the space of the Model S on the inside...,"[There, s, the, space, of, the, Model, S, on, ...",s space Model S inside far storage size constr...


In [ ]:
#df_no_musk = df_tesla[df_tesla["qids"] != "Q317521"]
df_tesla = df_tesla.drop_duplicates(subset = ['quotation'])
df_chevy = df_chevy.drop_duplicates(subset = ['quotation'])
df_tesla

,index,date,qids,quotation,quoteID,Cleaned Reviews,Tokenized,Stop Words Removed
0,0,2015-04-17 17:55:00,Q6751189,The policy will concentrate on `A' category st...,76279,The policy will concentrate on A category stat...,"[The, policy, will, concentrate, on, A, catego...",policy concentrate category stations developed...
1,1,2015-10-29 16:31:31,Q4934,As Sergey and I wrote in the original founders...,10100,As Sergey and I wrote in the original founders...,"[As, Sergey, and, I, wrote, in, the, original,...",Sergey wrote original founders letter years ag...
2,2,2015-10-14 10:18:21,Q317521,Elon Musk: Apple hired the Tesla engineers we'...,22047,Elon Musk Apple hired the Tesla engineers we v...,"[Elon, Musk, Apple, hired, the, Tesla, enginee...",Elon Musk Apple hired Tesla engineers ve fired
3,3,2015-05-21 18:50:00,Q21456834,"If Tesla folds tomorrow, there will be a few h...",28626,If Tesla folds tomorrow there will be a few hu...,"[If, Tesla, folds, tomorrow, there, will, be, ...",Tesla folds tomorrow people Connecticut disapp...
4,4,2015-10-12 02:46:53,Q29643422,This is the best racing on the planet in my op...,90436,This is the best racing on the planet in my op...,"[This, is, the, best, racing, on, the, planet,...",best racing planet opinion Late Model Stock Ca...
...,...,...,...,...,...,...,...,...
19572,934,2020-04-16 00:00:00,Q317521,Only requirement is that the vents are needed ...,36404,Only requirement is that the vents are needed ...,"[Only, requirement, is, that, the, vents, are,...",requirement vents needed immediately patients ...
19573,935,2020-02-19 09:18:25,Q317521,Gigafactory Berlin-Brandenburg is the next pha...,20355,Gigafactory Berlin Brandenburg is the next pha...,"[Gigafactory, Berlin, Brandenburg, is, the, ne...",Gigafactory Berlin Brandenburg phase Tesla s p...
19576,938,2020-01-15 13:46:36,Q125060,The state of California requires a lemon vehic...,84107,The state of California requires a lemon vehic...,"[The, state, of, California, requires, a, lemo...",state California requires lemon vehicle meet c...
19577,939,2020-03-19 00:00:00,Q317521,Tesla makes cars with sophisticated hvac syste...,59397,Tesla makes cars with sophisticated hvac syste...,"[Tesla, makes, cars, with, sophisticated, hvac...",Tesla makes cars sophisticated hvac systems Sp...


In [ ]:
vectorizer = TfidfVectorizer()

tfid = vectorizer.fit_transform(df_tesla["quotation"])
pairwise_similarity = (tfid * tfid.T).toarray()

duplicates_id = []
index = df_tesla.index

i = 0
for quote in pairwise_similarity:
    for val in range(i, len(quote)):
        if (quote[val] > 0.7) and (i != val):

            duplicates_id.append(index[val])
    i += 1

duplicates_id = list(set(duplicates_id))
duplicates_id.sort()

df_tesla = df_tesla.drop(duplicates_id, axis = 0)

In [ ]:
vectorizer = TfidfVectorizer()

tfid = vectorizer.fit_transform(df_chevy["quotation"])
pairwise_similarity = (tfid * tfid.T).toarray()

duplicates_id = []
index = df_chevy.index

i = 0
for quote in pairwise_similarity:
    for val in range(i, len(quote)):
        if (quote[val] > 0.7) and (i != val):

            duplicates_id.append(index[val])
    i += 1

duplicates_id = list(set(duplicates_id))
duplicates_id.sort()

df_chevy = df_chevy.drop(duplicates_id, axis = 0)

In [ ]:
df_tesla

,index,date,qids,quotation,quoteID,Cleaned Reviews,Tokenized,Stop Words Removed
0,0,2015-04-17 17:55:00,Q6751189,The policy will concentrate on `A' category st...,76279,The policy will concentrate on A category stat...,"[The, policy, will, concentrate, on, A, catego...",policy concentrate category stations developed...
1,1,2015-10-29 16:31:31,Q4934,As Sergey and I wrote in the original founders...,10100,As Sergey and I wrote in the original founders...,"[As, Sergey, and, I, wrote, in, the, original,...",Sergey wrote original founders letter years ag...
2,2,2015-10-14 10:18:21,Q317521,Elon Musk: Apple hired the Tesla engineers we'...,22047,Elon Musk Apple hired the Tesla engineers we v...,"[Elon, Musk, Apple, hired, the, Tesla, enginee...",Elon Musk Apple hired Tesla engineers ve fired
3,3,2015-05-21 18:50:00,Q21456834,"If Tesla folds tomorrow, there will be a few h...",28626,If Tesla folds tomorrow there will be a few hu...,"[If, Tesla, folds, tomorrow, there, will, be, ...",Tesla folds tomorrow people Connecticut disapp...
4,4,2015-10-12 02:46:53,Q29643422,This is the best racing on the planet in my op...,90436,This is the best racing on the planet in my op...,"[This, is, the, best, racing, on, the, planet,...",best racing planet opinion Late Model Stock Ca...
...,...,...,...,...,...,...,...,...
19570,932,2020-01-21 19:51:48,Q6990778,"Tesla's dominance in technology is real,",69758,Tesla s dominance in technology is real,"[Tesla, s, dominance, in, technology, is, real]",Tesla s dominance technology real
19571,933,2020-01-22 18:53:45,Q8538715,The settlement is technically only an agreemen...,87534,The settlement is technically only an agreemen...,"[The, settlement, is, technically, only, an, a...",settlement technically agreement Tesla Michigan
19573,935,2020-02-19 09:18:25,Q317521,Gigafactory Berlin-Brandenburg is the next pha...,20355,Gigafactory Berlin Brandenburg is the next pha...,"[Gigafactory, Berlin, Brandenburg, is, the, ne...",Gigafactory Berlin Brandenburg phase Tesla s p...
19576,938,2020-01-15 13:46:36,Q125060,The state of California requires a lemon vehic...,84107,The state of California requires a lemon vehic...,"[The, state, of, California, requires, a, lemo...",state California requires lemon vehicle meet c...


# VADER Analysis

In [ ]:
nltk.download('vader_lexicon')

def vadersentimentanalysis(review):

    vs = analyzer.polarity_scores(review)
    return vs['compound']

analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
df_tesla['Vader_Sentiment'] = df_tesla['Stop Words Removed'].apply(vadersentimentanalysis)
df_chevy['Vader_Sentiment'] = df_chevy['Stop Words Removed'].apply(vadersentimentanalysis)

# BERT Analysis

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
df_tesla['review'] = df_tesla['quotation'].apply(lambda x: sentiment_score(x[:512]))
df_chevy['review'] = df_chevy['quotation'].apply(lambda x: sentiment_score(x[:512]))

# Text 2 Emotion Analyis

In [ ]:
def high_emotion(d):

     v=list(d.values())
     k=list(d.keys())
     return k[v.index(max(v))]
     

In [ ]:
df_tesla['Text 2 Emotion'] = df_tesla['Stop Words Removed'].apply(te.get_emotion)
df_tesla['Emotion'] = df_tesla['Text 2 Emotion'].apply(high_emotion)

df_chevy['Text 2 Emotion'] = df_chevy['Stop Words Removed'].apply(te.get_emotion)
df_chevy['Emotion'] = df_chevy['Text 2 Emotion'].apply(high_emotion)

# Emotion Verification

In [ ]:
def vibe_check(vibe_BERT, vibe_VADER, vibe_t2e):

  vibe_curr = vibe_BERT

  if vibe_BERT == 1 or vibe_VADER == 2:
    if (vibe_VADER > 0.6) and (vibe_t2e == "Happy" or vibe_t2e == "Surprised"):
      vibe_curr = 4

  elif vibe_BERT == 4 or vibe_BERT == 5:
    if (vibe_VADER < 0.4) and (vibe_t2e == "Sad" or vibe_t2e == "Fear"):
      vibe_curr = 2
  
  return vibe_curr

In [ ]:
df_tesla['Vibe Checked'] = df_tesla.apply(lambda row: vibe_check(row['review'], row['Vader_Sentiment'], row['Text 2 Emotion']), axis=1)
df_chevy['Vibe Checked'] = df_chevy.apply(lambda row: vibe_check(row['review'], row['Vader_Sentiment'], row['Text 2 Emotion']), axis=1)

In [ ]:
df_tesla.to_json('Tesla_Sentiment_Everyone.json')
files.download('/content/Tesla_Sentiment_Everyone.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_chevy.to_json('Chevy_Sentiment_Everyone.json')
files.download('/content/Chevy_Sentiment_Everyone.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Selecting Frequent Speakers

In [ ]:
def emotion_count(dataframe):

  speakers = dataframe["qids"].drop_duplicates().tolist()

  pos_count = []
  neg_count = []
  neu_count = []
  tot_count = []

  speakers_group = dataframe.groupby("qids")

  for curr in speakers:
          
      analysis = speakers_group.get_group(curr)["review"]
      vals = Counter(analysis)
      pos_count.append(vals[5] + vals[4])
      neg_count.append(vals[1] + vals[2])
      neu_count.append(vals[3])
      tot_count.append(vals[1] + vals[2] + vals[3] + vals[4] + vals[5])
          
  emotion_count = pd.DataFrame()
  emotion_count["Speakers"] = speakers
  emotion_count["Positive Count"] = pos_count
  emotion_count["Negative Count"] = neg_count
  emotion_count["Neutral Count"] = neu_count
  emotion_count["Total Count"] = tot_count

  return emotion_count

In [ ]:
emotion_count_tesla = emotion_count(df_tesla)
most_cited = emotion_count_tesla[emotion_count_tesla["Total Count"] > 5]
most_cited_speakers = most_cited["Speakers"]
df_cited_tesla = df_tesla[df_tesla["qids"].isin(most_cited_speakers)]

In [ ]:
emotion_count_chevy = emotion_count(df_chevy)
most_cited = emotion_count_chevy[emotion_count_chevy["Total Count"] > 5]
most_cited_speakers = most_cited["Speakers"]
df_cited_chevy = df_chevy[df_chevy["qids"].isin(most_cited_speakers)]

In [ ]:
df_tesla.to_json('Tesla_Sentiment_Frequent.json')
files.download('/content/Tesla_Sentiment_Frequent.json')

emotion_count_tesla.to_json('Tesla_Speakers_Emotions.json')
files.download('/content/Tesla_Speakers_Emotions.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_chevy.to_json('Chevy_Sentiment_Frequent.json')
files.download('/content/Chevy_Sentiment_Frequent.json')

emotion_count_chevy.to_json('Chevy_Speakers_Emotions.json')
files.download('/content/Chevy_Speakers_Emotions.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
df_tesla = pd.read_json('/content/drive/MyDrive/final_dataframes/Chevy_Sentiment_Frequent.json')
df_chevy = pd.read_json('/content/drive/MyDrive/final_dataframes/Tesla_Sentiment_Frequent.json')

# Load Relevant Supplementary Data

In [4]:
from_qids_to_label_occupation = pd.read_json('/content/drive/MyDrive/speaker_attributes/occupations.json') 
from_qids_to_label_ethnics = pd.read_json('/content/drive/MyDrive/speaker_attributes/ethnic.json' ) 
from_qids_to_label_religion = pd.read_json('/content/drive/MyDrive/speaker_attributes/religion.json') 
from_qids_to_label_nationality = pd.read_json('/content/drive/MyDrive/speaker_attributes/nationality.json') 
from_qids_to_label_party = pd.read_json('/content/drive/MyDrive/speaker_attributes/party.json') 
from_qids_to_label_gender = pd.read_json('/content/drive/MyDrive/speaker_attributes/gender.json') 

df_speakers_attributes = pd.read_parquet('/content/drive/MyDrive/speaker_attributes/speaker_attributes.parquet')

# Map Speaker Attributes

In [14]:
def map_qids_labels(list_qids,df_):
    """Transform a list of QIDS into a list of labels, return 0 if the list is empty (which is equal to 0 in the dataframe)."""
    if(type(list_qids)!=int):
        return [df_[df_.qids == x].values[0][1] for x in list_qids if (x!=0 and len(df_[df_.qids == x].values) != 0)]    
    else:
        return 0  

def process_speakers_attributes(df_speakers_attributes):
    """Add the columns for the party, occupation, religion, ethnics and nationality their labels. Take as input the dataframe with the selected speakers and return the same dataframe with the columns added."""
    df = df_speakers_attributes[['date_of_birth','nationality','gender','ethnic_group','occupation','id','label','religion','party']]
    df = df.rename(columns={'date_of_birth':'birth','nationality':'nationality_qids','gender':'gender_qids','ethnic_group':'ethnics_qids','occupation':'occupations_qids','id':'name_qids','label':'name_label','religion':'religion_qids','party':'party_qids','gender':'gender_qids'})
    # Add labels for occupations, ethincs, religion, nationality, party
    df = df.fillna(0)
    df['occupation_label']= df['occupations_qids'].apply(lambda x : map_qids_labels(x,from_qids_to_label_occupation))
    df['ethnics_label']= df['ethnics_qids'].apply(lambda x : map_qids_labels(x,from_qids_to_label_ethnics))
    df['religion_label']= df['religion_qids'].apply(lambda x : map_qids_labels(x,from_qids_to_label_religion))
    df['nationality_label']= df['nationality_qids'].apply(lambda x : map_qids_labels(x,from_qids_to_label_nationality))
    df['party_label']= df['party_qids'].apply(lambda x : map_qids_labels(x,from_qids_to_label_party))
    df['gender_label']= df['gender_qids'].apply(lambda x : map_qids_labels(x,from_qids_to_label_gender))
    return df


In [9]:
def select_speakers(list_qids):
  
    """From a list of QIDS of speakers, return the dataframe with only the speakers that are in the list."""
    return df_speakers_attributes[df_speakers_attributes.id.isin(list_qids)]

def process_dataframe_quotes(df_sent, time_range, speakers, explode_by):
    """ Transform the date into a datetime object ('%Y-%m-%d %H:%M:%S)
        Split with the time_range the dataframe df. Add a columns 'period' with the label before, during or after. 
        Join the quotes dataframe with the speakers dataframe on the name_qids.
        Explode the dataframe depending on the column(s) in explode_by"""
    
    # Rename the columns and keep only the one needed
    df_sent = df_sent.rename(columns={'qids':'name_qids','quoteID':'quote_qids','Emotion':'emotion'})
    df_sent = df_sent[['date','name_qids','quotation','quote_qids','Vader_Sentiment','emotion','review']]
    # Change timestamp in datetime object and add the perdio depending on the time_range list
    df_sent['date']= df_sent['date'].apply(lambda x :datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S'))
    df_sent['year_month']= df_sent['date'].apply(lambda x :datetime.strptime(str(x)[:7], '%Y-%m'))
    
    #df_sent['period'] = df_sent['date'].apply(lambda x : select_period(x,time_range))
    
    #Join the dataframe with the speakers attributes on the name_qids
    df = df_sent.merge(speakers, on='name_qids')

    # Explode the df by the column(s) in explode_by
    #df = df.explode(explode_by)
    
    return df

In [10]:
list_qids = df_tesla.drop_duplicates(['qids']).qids.values
df_tesla_ = select_speakers(list_qids)
df_speakers_att_tesla = process_speakers_attributes(df_tesla_)

In [15]:
list_qids = df_chevy.drop_duplicates(['qids']).qids.values
df_chevy_ = select_speakers(list_qids)
df_speakers_att_chevy = process_speakers_attributes(df_chevy_)

# Process Dates

In [20]:
start = datetime.strptime(str('2017-07-28 00:00:00'), '%Y-%m-%d %H:%M:%S')
end = datetime.strptime(str('2017-07-28 23:59:59'), '%Y-%m-%d %H:%M:%S')
time_range = [start,end]
speakers = df_speakers_att_tesla[['name_qids','name_label','occupations_qids']]
explode_by = 'occupations_qids'
df_tesla = process_dataframe_quotes(df_tesla, time_range, speakers, explode_by)

In [21]:
start = datetime.strptime(str('2017-07-28 00:00:00'), '%Y-%m-%d %H:%M:%S')
end = datetime.strptime(str('2017-07-28 23:59:59'), '%Y-%m-%d %H:%M:%S')
time_range = [start,end]
speakers = df_speakers_att_chevy[['name_qids','name_label','occupations_qids']]
explode_by = 'occupations_qids'
df_chevy = process_dataframe_quotes(df_chevy, time_range, speakers, explode_by)

In [22]:
df_tesla

,date,name_qids,quotation,quote_qids,Vader_Sentiment,emotion,review,year_month,name_label,occupations_qids
0,2015-02-04 05:35:55,Q5178606,I've had to go back 10 years of my program fin...,53298,0.0000,Surprise,5,2015-02-01,Courtney Force,[Q378622]
1,2015-10-31 09:27:41,Q5178606,Robert Hight stepped up with Mike Neff and tha...,50268,0.0000,Happy,5,2015-10-01,Courtney Force,[Q378622]
2,2015-07-25 11:08:06,Q5178606,I saw (crew chief Jon) Schaffer running for th...,21150,0.9834,Happy,4,2015-07-01,Courtney Force,[Q378622]
3,2015-01-25 11:31:30,Q5178606,We have a lot of talent here. We had a lot of ...,48224,0.9201,Surprise,5,2015-01-01,Courtney Force,[Q378622]
4,2015-09-07 11:03:37,Q5178606,To be here in the heat in Indy coming from whe...,66860,0.8720,Happy,4,2015-09-01,Courtney Force,[Q378622]
...,...,...,...,...,...,...,...,...,...,...
3505,2020-01-25 20:20:39,Q1252109,A 2010 Chevrolet Silverado was traveling eastb...,401,0.1531,Fear,1,2020-01-01,Doug Clark,[Q484188]
3506,2020-01-23 21:19:54,Q188235,She ain't got no use for a BMW or wine from a ...,72735,0.0000,Sad,2,2020-01-01,Travis Tritt,"[Q488205, Q177220, Q753110, Q33999]"
3507,2020-01-16 02:24:00,Q6766762,"If you look at Chevrolet's lineup,",40235,0.0000,Happy,4,2020-01-01,Mark Block,0
3508,2020-03-11 18:49:06,Q6279,"My dad didn't have a lot of money, but he ran ...",43088,0.0000,Sad,5,2020-03-01,Joe Biden,"[Q82955, Q40348, Q1622272]"


In [ ]:
df_tesla.to_json('Tesla_Sentiment_Complete.json')
files.download('/content/Tesla_Sentiment_Complete.json')

In [ ]:
df_tesla.to_json('Tesla_Sentiment_Complete.json')
files.download('/content/Tesla_Sentiment_Complete.json')

In [ ]:
test = df_sent.explode('occupations_qids')
test